In [ ]:
from my_module import np, h5py, os, plt


WORKING_DIR = "/u/sciteam/smzyz/scratch/results/MOD_training/_500X500"

inum = 100
cloudy_file = "cloudy_batch_{}.h5".format(inum)
clear_file = "batch_{}.h5".format(inum)

with h5py.File(os.path.join(WORKING_DIR, clear_file), 'r') as h5f:
    data_all = h5f['X'][:]

In [ ]:

data = np.rollaxis(data_all[100, :, :, [0, 3, 2]]/300., 0, 3)
plt.imshow(data)
plt.show()


# 2018.05.10

Test MOD35 downloader

In [ ]:
from my_module import np, os, tqdm, sys
import csv
import shutil
import ssl
import urllib2
# import mpi4py.MPI as MPI



USERAGENT = 'tis/download.py_1.0--' + sys.version.replace('\n','').replace('\r','')


def url_retrieve(src, dst):
    """
    Return 0 if an error occur otherwise Return 1.
    """
    
    url = src
    headers = { 'user-agent' : USERAGENT }
    headers['Authorization'] = 'Bearer ' + "59451810-53C9-11E8-9F28-C61EAE849760"
    CTX = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    
    try:
        fh = urllib2.urlopen(urllib2.Request(url, headers=headers), context=CTX)
        shutil.copyfileobj(fh, dst)
    except urllib2.HTTPError as e:
        print '>> HTTPError: {}'.format(url)
    except urllib2.URLError as e:
        print '>> URLError: {}'.format(url)


In [94]:

def main(iyr, iday):
    iday = str(iday).zfill(3)
    
    # Set up download folder (dst_parent_folder)
    dst_parent_folder = "/u/sciteam/smzyz/scratch/data/MODIS/MOD35/{}".format(iyr)
    dst_daily_folder = os.path.join(dst_parent_folder, iday)
    try:
        print ">> log: {}".format(dst_daily_folder)
        os.mkdir(dst_daily_folder)
    except OSError as err:
        pass
    
    
    # Set up server folder (src_parent_folder)
    src_parent_folder = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD35_L2/{}'.format(iyr)
    src_daily_folder = os.path.join(src_parent_folder, iday)
    
    
    # 1) Download daily .csv file from (src_parent_folder) to (dst_parent_folder)
    #    e.g., src: https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD35_L2/2000/300.csv
    #          dst: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/300.csv
    src_csv_file = "{}/{}.csv".format(src_parent_folder, iday)
    dst_csv_file = "{}/{}.csv".format(dst_parent_folder, iday)
    with open(dst_csv_file, 'w+b') as out:
        url_retrieve(src_csv_file, out)
    
    
    # 2) Read csv file and generate a download file list by cross-referencing existed MOD35 files
    files_server = [ f['name'] for f in csv.DictReader(open(dst_csv_file), skipinitialspace=True) ]
#     sizes_server = [ f['size'] for f in csv.DictReader(open(dst_csv_file), skipinitialspace=True) ]
    files_local = os.listdir(dst_daily_folder)
#     sizes_local = [os.path.getsize(os.path.join(dst_daily_folder, ifile)) for ifile in files_local]
    files_new = [ifile for ifile in files_server if ifile not in files_local]

    if len(files_new) == 0:
        return
    else:
        src_file_list = ["{}/{}".format(src_daily_folder, f) for f in files_new]
        dst_file_list = ["{}/{}".format(dst_daily_folder, f) for f in files_new]
        
    
        # 3) Download
        print ">> Download: {}: {} files".format(dst_daily_folder, len(files_new))
        for i in range(len(src_file_list)):
            isrc = src_file_list[i]
            idst = dst_file_list[i]
            with open(idst, 'w+b') as out:
                url_retrieve(isrc, out)
            
    
for iday in range(54, 367):
    main(2000, iday)

>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/054
>> HTTPError: https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD35_L2/2000/054.csv
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/055
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/056
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/057
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/058
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/059
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/060
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/061
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/062
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/063
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/064
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/065
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/066
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/067
>> log: /u/sciteam/smzyz/scratch/data/MODIS/MOD35/2000/068
>> log: /u/sciteam/sm

KeyboardInterrupt: 

In [97]:
from my_module.data.modis import MOD35_retrieve_cloud_mask

flag_cloud_mask = MOD35_retrieve_cloud_mask("/u/sciteam/smzyz/scratch/data/MODIS/MOD35/2006/064/MOD35_L2.A2006064.0000.061.2017261111758.hdf")

In [99]:
flag_cloud_mask.shape

(2030, 1354, 7)